In [213]:
using SparseArrays, LinearAlgebra

function readgraph(graph)
    I=Int[]
    J=Int[]
    open(graph) do f
        linecounter = 0
        for l in eachline(f)
            v = split(l)
            if v[1] == "D"
                i,j = extrema((parse(Int, v[2])+1, parse(Int, v[3])+1))
                push!(I, i)
                push!(J, j)
            end
        end
    end

    N = maximum(I)
    H = sparse(J.-N,I,fill(1,length(I))) .÷ 2
end

function readseeds(seeds, H)
    M, N = size(H)
    @show size(H)
    v = Int[]
    open(seeds) do f
        for l in eachline(f)
            if l[1] != '#'
                v = parse.(Int, split(l)) .- N .+ 1
            end
        end
        println("$(length(v)) removed factors $v")
        return H[setdiff(1:size(H,1), v), :]
    end
end

readseeds (generic function with 1 method)

In [287]:
function leaf_removal(H::SparseMatrixCSC)
    M, N = size(H)
    Ht = sparse(transpose(H)) # for faster access by row
    degs = vec(sum(H .!= 0, dims=1))
    facts = fill(true, M)
    rowperm = Int[]
    Q = findall(degs .== 1)
    indep = findall(degs .== 0)
    dep = Int[]
    while !isempty(Q)
        i = popfirst!(Q)
        degs[i] == 0 && continue
        push!(dep, i)
        ∂i = rowvals(H)[nzrange(H,i)]
        ∂i = ∂i[facts[∂i]]
        @assert length(∂i) == 1 # should be a residual leaf
        a = ∂i[1]
        facts[a] = false
        push!(rowperm, a) 
        for j in rowvals(Ht)[nzrange(Ht,a)]
            degs[j] -= 1
            if j != i
                if degs[j] == 0
                    push!(indep, j)
                elseif degs[j] == 1
                    push!(Q, j)
                end
            end
        end
    end
    all(degs .==  0) || @warn "non-empty core"
    rowperm, [dep; indep]
end

leaf_removal (generic function with 1 method)

In [328]:
rowperm, colperm = leaf_removal(H)

([6226, 12140, 9470, 8649, 6874, 7504, 6361, 9440, 1080, 5996  …  6783, 5508, 10322, 10387, 7461, 9724, 9686, 12924, 1855, 1943], [10, 101, 207, 256, 282, 304, 320, 337, 357, 449  …  10977, 13169, 675, 13612, 11768, 13904, 14969, 15333, 11772, 12451])

In [329]:
H1=H[rowperm, colperm]

13532×17004 SparseMatrixCSC{Int64,Int64} with 40595 stored entries:
⠙⢿⣾⣿⣿⣿⣷⣿⣿⣾⣾⣾⣿⣶⡾⣿⣾⣞⢿⣿⣳⣺⣽⣯⣟⣵⣾⡳⢵⣿⣯⣾⣿⣺⡛⣛⡟⣿⡮⢲
⠀⠀⠙⢿⣿⣿⡿⣿⣿⣿⣿⣿⡿⣿⣽⣿⣯⣿⣿⡹⣿⡿⣻⣿⣾⣗⣿⣻⣿⠿⣿⣻⣿⡿⣿⣿⣲⡦⡏⣿
⠀⠀⠀⠀⠙⢿⣿⣿⣿⣿⣿⣿⣯⢿⣿⡽⣿⣿⣿⣿⣿⣿⣯⣯⣷⣻⣷⣿⢗⣻⢾⣽⣟⣿⣿⣿⣻⣾⣷⠧
⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⢿⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣺⣷⣿⣿⣿⣿⡿⣿⣿⣾⣻⣿⣿⣾⣟⣿⢶⣿⢇
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢯⡻⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⡿⣿⣿⣿⣽⡟⣿⣽⣿⣜⢻⣿⣿⣷⣽⡿⣿⢾⣻
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢮⡛⢿⣿⣿⣿⣿⣾⣿⣿⣿⣯⣿⣿⣻⣿⣿⣷⣿⢿⣽⣿⣻⣿⣿⣷⣿⣿⣗
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡙⠿⣷⣿⣿⡿⣿⣿⣻⣯⣿⣯⡿⣷⣿⡿⡿⡟⣿⣿⣯⣿⣯⣿⣾⡟
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡈⠻⣿⣿⣾⣿⣿⣾⣾⣿⣿⠿⣿⣿⣿⡗⣿⣟⣿⣿⣻⡳⣟⠍
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠙⢿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⣷⢹⣿⣻⣿⣿⡿⣟⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠙⠻⣿⣷⣿⣟⣿⣿⣯⣿⡟⠸⣿⣿⣿⡿⣿⣻⢣
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠻⣿⣷⣿⣿⣷⣿⣿⠀⢿⣿⣿⣿⣿⣿⣽
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠻⣿⣻⣿⣾⣟⠀⠘⣿⣿⣿⣮⣾⢾
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠻⣷⣟⣾⠀⠀⠹⣿⣿⣿⣿⡯
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠙⣿⣦⠀⠀⠀⠹⣿⣿⣿⣟
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡈⣿⠀⠀⠀⠀⠘⢿⣿⡿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠾⠀⠀⠀⠀⠀⠀⠙⠿

In [330]:
length(H1)

230098128

In [314]:
ut2diagGF2!(H1)

13532×17004 SparseMatrixCSC{Int64,Int64} with 450877 stored entries:
⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⣿⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⣿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠘⣿⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠹⣿⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠹⣿⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠘⢿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠦⠀⠀⠀⠀⠀⠀⠙⠿

In [312]:
isuppertriang(H::SparseMatrixCSC) = all(rowvals(H)[last(nzrange(H,i))] == i for i = 1:size(H,1))

function ut2diagGF2!(T::SparseMatrixCSC)
    (m,n) = size(T)
    # Check that the left part of T is unit upper triangular
    @assert isuppertriang(T)
    # Loop over diagonal elements
    for c in m:-1:2
        # Find non-zero elements above T[c,c] and perform row operations to 
        #  cancel them out
        for (j,v) in @views zip(rowvals(T[:,c]),nonzeros(T[:,c]))
            if v != 0 && j < c
                for k in rowvals(T[c,c:end]).+c.-1
                    T[j,k] ⊻= T[c,k]
                end
            end
        end
    end
    dropzeros!(T)
end

ut2diagGF2! (generic function with 1 method)

In [132]:
function findbasis(H)
    rowperm, colperm = leaf_removal(H)
    Hnew = H[rowperm, colperm]
    ut2diagGF2!(Hnew)
    B = [Hnew[:, size(Hnew, 1)+1:end]; I]
    indep = colperm[size(H,1)+1:end]
    B[invperm(colperm),:], indep
end 

findbasis (generic function with 1 method)

In [364]:
H = readgraph("/home/ab/src/spread/graph.txt")
H = readseeds("/home/ab/src/spread/seeds.txt", H)
B, indep = findbasis(H);
M,N = size(H)
α=1-M/N

size(H) = (13670, 17004)
138 removed factors [37, 135, 155, 252, 576, 612, 835, 1020, 1070, 1117, 1230, 1581, 1608, 1761, 1962, 2181, 2185, 2383, 2416, 2503, 2594, 2722, 2743, 2951, 3023, 3243, 3244, 3327, 3350, 3361, 3364, 3442, 3504, 3548, 3707, 3710, 3784, 4017, 4055, 4133, 4162, 4205, 4284, 4314, 4328, 4511, 4544, 4650, 4706, 4744, 5012, 5068, 5170, 5238, 5315, 5559, 5777, 5789, 5874, 5901, 6049, 6086, 6251, 6334, 6342, 6488, 6509, 6586, 6635, 6710, 6733, 6802, 6858, 6918, 7100, 7199, 7276, 7280, 7311, 7579, 7813, 8054, 8157, 8478, 8643, 8695, 8753, 9010, 9265, 9356, 9489, 9569, 9709, 9815, 9874, 9924, 9936, 9962, 10109, 10210, 10302, 10312, 10551, 10686, 10843, 10921, 11042, 11063, 11106, 11195, 11210, 11273, 11316, 11323, 11357, 11408, 11470, 11585, 11682, 11838, 11922, 12029, 12243, 12272, 12282, 12312, 12521, 12620, 12680, 12822, 12844, 13039, 13198, 13358, 13359, 13467, 13599, 13663]


0.20418725005880967

In [361]:
using JLD
save("/home/ab/src/sourcecoding/matrix-stefano2.jld", "H", H)

In [11]:
include("/home/ab/src/sourcecoding/code/headers.jl")

In [51]:
ms = MS(maxiter=10000, convergence=:decvars, Tmax=1, gamma=1e-3, sigma=1e-8)
M,N = size(H)
@show 1-M/N
fg = FactorGraphGF2(H)
ProgressMeter.ijulia_behavior(:clear)

1 - M / N = 0.24002849002849003


false

In [358]:
using SparseArrays, Random

struct MaxSum{V,F}
    H :: SparseMatrixCSC{V,Int}
    M :: SparseMatrixCSC{F,Int}
    mes :: Vector{F} 
    belief :: Vector{F}
    efield :: Vector{F}
end

function maxsum(H,efield)
    H = sparse(transpose(H))
    mes = zeros(eltype(efield), length(H.nzval))
    #we alias structure from H
    M = SparseMatrixCSC(size(H)...,H.colptr,H.rowval,mes)
    MaxSum(H, M, mes, copy(efield), copy(efield))
end

function update!(ms::MaxSum, f)
    maxchange = zero(eltype(ms.belief))
    fmin, fmin2, imin = Inf, Inf, 1
    s = 1.0
    # Loop over neighbors of `f`, computing:
    # - prod of signs `s`
    # - first min (and index) and second min of abs values of messages
    ∂(f) = nzrange(ms.M,f)
    for k in ∂(f)
        mo, i = ms.mes[k], ms.M.rowval[k]
        # subtract out message from belief
        mi = (ms.belief[i] -= mo)
        s *= sign(mi)
        am = abs(mi)
        if fmin > am
            fmin, fmin2, imin = am, fmin, i
        elseif fmin2 > am
            fmin2 = am
        end 
    end
    for k in ∂(f)
        mo, i = ms.mes[k], ms.M.rowval[k]
        # update message
        mnew = (i == imin ? fmin2 : fmin) * s * sign(ms.belief[i])
        maxchange = max(maxchange, abs(mnew - mo))
        ms.mes[k] = mnew
        # update belief after updating the message
        ms.belief[i] += mnew
    end 
    maxchange
end


function iteration!(ms::MaxSum; maxiter = 1000, tol=1e-3, γ=0.0, callback=(x...)->nothing)
    perm = collect(1:M)
    @inbounds for t = 1:maxiter        
        ms.belief .= ms.efield .+ vec(sum(ms.M, dims=2))
        maxchange = maximum(update!.(tuple(ms), shuffle!(perm)))
        ms.efield .+= γ.*ms.belief
        if maxchange < tol
            break
        end
        callback(t, ms)
    end
end                 


iteration! (generic function with 1 method)

In [365]:
using ProgressMeter
callback(t,ms) = nothing;#println("$t $(ms.belief[1:4])")

nsamples = 10
ms = maxsum(H,  1 .- 2y .+ 1e-8*randn(N));

# the mapping is (0,1)=>(1,-1)
dist = map(1:nsamples) do i
    y = rand((0,1), N)
    ms.M .= 0
    ms.efield .= 1 .- 2y .+ 1e-8*randn(N)
    iteration!(ms; maxiter=2000, γ=1e-3, callback=callback)
    x = Int.(ms.belief .< 0)
    c = sum(ms.H'x .% 2)
    d = sum(x .⊻ y)/N
    @show d c
    x1 = B*x[indep] .% 2; d1 = sum(x1 .⊻ y)/N; c1 = sum(ms.H'x1 .% 2); @show d1 c1 
    d1
end
sum(dist)/nsamples

d = 0.26323218066337334
c = 0
d1 = 0.26323218066337334
c1 = 0
d = 0.2633498000470478
c = 2
d1 = 0.2635262291225594
c1 = 0
d = 0.26329099035521053
c = 0
d1 = 0.26329099035521053
c1 = 0
d = 0.26329099035521053
c = 5
d1 = 0.2646436132674665
c1 = 0
d = 0.2642907551164432
c = 5
d1 = 0.26523171018583863
c1 = 0
d = 0.265055281110327
c = 0
d1 = 0.265055281110327
c1 = 0
d = 0.2627028934368384
c = 1
d1 = 0.26299694189602446
c1 = 0
d = 0.26523171018583863
c = 0
d1 = 0.26523171018583863
c1 = 0
d = 0.2646436132674665
c = 26
d1 = 0.27111267936956013
c1 = 0
d = 0.26534932956951307
c = 6
d1 = 0.26940719830628085
c1 = 0


0.26537285344624795

In [366]:
1-M/N,sum(dist)/nsamples

(0.20418725005880967, 0.26537285344624795)

In [283]:
x=[1:10;]; f=(x .% 2 .== 1);x[findall(f)]==x[f]

true

In [52]:
dist = map(1:10) do i
    Random.seed!(i)
    
    y = rand((0,1), N)

    lm = LossyModelGF2(fg, zeros(N), Inf, 1.0, y)
    
    function checker()
        v = (B*((fg.fields .< 0)[indep])) .% 2;
        dist = sum(v .⊻ y)/N
        check = sum(H*(fg.fields .< 0) .% 2)
        check, dist
    end

    independent = falses(N)
    independent[indep] .= true
    res_MS = solve!(lm, ms, randseed=1, verbose=true, showprogress=true, 
        independent=independent, basis = B)
#    res_MS = solve!(lm, ms, randseed=1, verbose=true, showprogress=true)
    avg_dist = res_MS.distortion 
    check, dist = checker()
    @show dist check avg_dist
    dist
end

Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.24287749287749288
check = 1
avg_dist = 0.4309116809116809


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.24786324786324787
check = 1
avg_dist = 0.4309116809116809


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


...

Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.245014245014245
check = 6
avg_dist = 0.4458689458689459
dist = 0.24715099715099714
check = 0
avg_dist = 0.24715099715099714


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.25427350427350426
check = 2
avg_dist = 0.44515669515669515
dist = 0.24857549857549857
check = 0
avg_dist = 0.24857549857549857


10-element Vector{Float64}:
 0.24287749287749288
 0.24786324786324787
 0.25427350427350426
 0.26495726495726496
 0.2378917378917379
 0.26851851851851855
 0.245014245014245
 0.24715099715099714
 0.25427350427350426
 0.24857549857549857

In [53]:
dev = sqrt(sum(dist.^2)/length(dist)-(sum(dist)/length(dist))^2)/sqrt(length(dist))
1-M/N, sum(dist)/length(dist)

(0.24002849002849003, 0.2511396011396011)

In [31]:
1-M/N, 0.318686262747450

(0.24002849002849003, 0.31868626274745)

In [102]:
sum(H*(fg.fields .< 0) .% 2)

1

In [70]:
cw = fg.fields .< 0;
dist = sum(cw .!= y)/N

0.2167393675027263

In [71]:
1-M/N, dist

(0.30552526354053067, 0.2167393675027263)

In [707]:
B,indep=findbasis(H)

(
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣛⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⠛⣽⣿⣷⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⢬⢿⣿⣿⢿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇, [1305, 5928, 239, 2177, 2836, 2337, 3619, 6060, 943, 9471  …  1620, 10099, 3460, 10249, 7536, 10732, 3862, 5283, 2205, 6647])

In [708]:
B

11004×3362 SparseMatrixCSC{Int64,Int64} with 71353 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣛⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⠛⣽⣿⣷⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⢬⢿⣿⣿⢿⢿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇

In [233]:
H .= 0

13469×17004 SparseMatrixCSC{Int64,Int64} with 40406 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⢽⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣯⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⠿⠿⠿⠿⠻⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿

In [540]:
isuppertriang(Hnew)

true

In [506]:
findbasis(H)

LoadError: AssertionError: isuppertriang(T)

In [521]:
    rowperm, colperm = leaf_removal(H)
    Hnew = H[rowperm, colperm]


7642×11004 SparseMatrixCSC{Int64,Int64} with 22925 stored entries:
⠙⢾⣬⣐⢄⡖⢐⡌⣄⢌⡥⢊⢹⡀⣒⢂⠀⠘⠈⠀⢒⣰⢠⢊⢘⣎⠕⣸⡰⣀⡔⣢⠠⠥⠘⡀⢒⡲⡀⡠
⠀⠀⠙⢮⡓⠦⣇⠀⠁⣦⠨⣕⠨⢞⡏⢈⣤⣄⢅⠀⠼⡣⡇⠀⠰⢊⠋⢼⡄⠄⡄⠘⣳⢋⠇⠺⢤⠡⠄⣉
⠀⠀⠀⠀⠙⢦⡈⠙⠾⣦⢨⡤⠨⠮⢈⠗⣊⢁⢁⡐⣣⠦⢐⢲⡥⠌⣋⠺⣧⠘⡄⣪⠐⢕⠋⠦⡠⠄⠫⡤
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠙⠶⣥⢪⠆⠠⡝⢒⠱⠅⣨⡲⠠⠈⢏⠉⣤⠁⣿⣯⣍⠍⢄⣺⣄⠪⡃⣁⠔⡂
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⠙⢧⣆⣐⡯⣼⡪⡴⠛⢒⡼⠂⡟⡠⠆⢻⣷⣮⡛⡄⠕⠈⢇⠑⠴⡆⢇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⠳⣾⢄⢊⡘⢱⡚⡥⣠⣉⢽⠁⠸⣿⣿⣆⢒⢆⢿⡕⠧⡵⡢⢦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⠙⢧⡍⡲⠮⡊⠂⣳⣱⠩⠀⢿⣿⣿⣮⠱⡊⡎⠨⡀⡡⠈
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠙⢷⣵⢵⢓⢚⠓⣇⠀⠈⣿⣿⣿⣯⠘⠶⠕⠅⢤⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠙⣇⠜⡱⠐⣂⠀⠀⠘⣿⣿⣿⣿⣥⠧⡮⢜⡁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⢻⣥⣜⡆⠀⠀⠀⠘⢿⣿⣿⣿⣝⣱⢼⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠹⡼⠧⠀⠀⠀⠀⠈⢿⣿⣿⣿⣶⣅⣲
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠹⡖⠀⠀⠀⠀⠀⠀⠙⣿⣿⣿⣧⠼
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⢯⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠾⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠿

In [525]:
isuppertriang(Hnew)

false

In [523]:
dropzeros!(Hnew)

7642×11004 SparseMatrixCSC{Int64,Int64} with 22925 stored entries:
⠙⢾⣬⣐⢄⡖⢐⡌⣄⢌⡥⢊⢹⡀⣒⢂⠀⠘⠈⠀⢒⣰⢠⢊⢘⣎⠕⣸⡰⣀⡔⣢⠠⠥⠘⡀⢒⡲⡀⡠
⠀⠀⠙⢮⡓⠦⣇⠀⠁⣦⠨⣕⠨⢞⡏⢈⣤⣄⢅⠀⠼⡣⡇⠀⠰⢊⠋⢼⡄⠄⡄⠘⣳⢋⠇⠺⢤⠡⠄⣉
⠀⠀⠀⠀⠙⢦⡈⠙⠾⣦⢨⡤⠨⠮⢈⠗⣊⢁⢁⡐⣣⠦⢐⢲⡥⠌⣋⠺⣧⠘⡄⣪⠐⢕⠋⠦⡠⠄⠫⡤
⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠙⠶⣥⢪⠆⠠⡝⢒⠱⠅⣨⡲⠠⠈⢏⠉⣤⠁⣿⣯⣍⠍⢄⣺⣄⠪⡃⣁⠔⡂
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⠙⢧⣆⣐⡯⣼⡪⡴⠛⢒⡼⠂⡟⡠⠆⢻⣷⣮⡛⡄⠕⠈⢇⠑⠴⡆⢇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⠳⣾⢄⢊⡘⢱⡚⡥⣠⣉⢽⠁⠸⣿⣿⣆⢒⢆⢿⡕⠧⡵⡢⢦
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⠙⢧⡍⡲⠮⡊⠂⣳⣱⠩⠀⢿⣿⣿⣮⠱⡊⡎⠨⡀⡡⠈
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠙⢷⣵⢵⢓⢚⠓⣇⠀⠈⣿⣿⣿⣯⠘⠶⠕⠅⢤⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠙⣇⠜⡱⠐⣂⠀⠀⠘⣿⣿⣿⣿⣥⠧⡮⢜⡁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠈⢻⣥⣜⡆⠀⠀⠀⠘⢿⣿⣿⣿⣝⣱⢼⣄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠹⡼⠧⠀⠀⠀⠀⠈⢿⣿⣿⣿⣶⣅⣲
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠹⡖⠀⠀⠀⠀⠀⠀⠙⣿⣿⣿⣧⠼
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⢯⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠾⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠿

In [531]:
Hnew[:,1:size(Hnew,1)]

7642×7642 SparseMatrixCSC{Int64,Int64} with 15740 stored entries:
⠙⢷⢮⣐⠐⠀⡀⡐⠂⠔⡐⡀⠀⠠⢉⡀⠒⠈⡆⠀⠠⠄⠂⠀⠀⠌⠐⠀⠀⠔⠀⢐⢀⠠⠁⠸⡂⠡⠅⢀
⠀⠀⠑⢌⠙⠦⣔⡂⠀⡍⠈⠢⢁⠀⠋⠰⢐⠀⠊⡡⡬⠆⠅⠀⠀⠐⠀⠀⠀⠆⠩⠐⡁⠈⠀⠐⡪⢡⠰⠐
⠀⠀⠀⠀⠑⢄⠀⠙⠲⣅⡀⠀⢀⠳⠂⡂⡆⠀⠒⠨⠆⠀⡠⢒⣈⣀⠂⠄⠈⡈⢋⠱⠃⠀⠀⠑⠠⡀⡀⠡
⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠙⠲⣧⡠⠀⡄⡠⠀⣘⠪⠀⠩⡠⠂⠁⠀⢀⠀⠌⠂⢴⠀⠐⢐⢌⠠⡀⠐⠠⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠙⠲⣬⣅⠂⠀⠤⢀⠁⠀⠮⡀⡁⢄⠨⠐⠐⠄⡀⠁⠈⢔⠑⠠⠔⠃⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠙⠺⣅⠘⠈⠀⠐⠔⠀⣀⠈⡈⢀⠪⠡⠐⠘⠀⡐⠤⣀⠜⢁⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠈⠙⢧⣐⡀⠐⡉⠠⡠⠆⣀⡰⠍⠁⢇⣔⠆⠂⡓⠀⠠⠆
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠈⠳⣍⣨⡐⠀⢡⠐⠰⠤⡄⡤⠂⠀⠐⠈⠊⠰⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠙⢦⣂⢄⠴⢀⠈⠐⡂⠊⡃⠶⠲⠐⠅⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠙⠳⣄⠅⢙⠰⠰⣐⠈⠀⠱⠈⣃⠅
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠈⠳⣪⣐⢄⠠⣠⠄⡉⠵⠔⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠙⢮⡌⠵⠁⢴⠀⠄⠀⠓
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠙⢦⡀⠌⡐⠁⠉⣀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠀⠳⡎⠐⡀⢆⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠀⠙⣮⠪⡜⠅
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠀⠘⣗⠓⠤
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠀⠘⣦⠴
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⠀⠘⡧
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢄⠀⢳
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠑⢼

In [536]:
for i=1:size(Hnew,1)
    if rowvals(Hnew)[last(nzrange(Hnew,i))] > i
        println(i, " ", nzrange(Hnew,i))
    end
end

In [533]:
Hnew[70:80,70:80]

11×11 SparseMatrixCSC{Int64,Int64} with 11 stored entries:
 1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1

In [519]:
nzrange(Hnew,1)

1:1

In [617]:
f(x)=(i=1;println(x))

f (generic function with 1 method)

In [619]:
f("ciao $i")

LoadError: UndefVarError: i not defined

In [148]:
?GaloisField

search: GaloisField GaloisFields @GaloisField @GaloisField!



```
const F = GaloisField(p)
const F,α = GaloisField(p, :β => [1, 0, 1])
const F,α = GaloisField(p, n, :β)
```

Return a type representing a finite field.

The single-argument signature returns the finite field $ℤ/pℤ$.

The two-arguments signature returns an algebraic extension of that field, with minimum polynomial given by the second argument: a dense representation of the univariate, monic polynomial, with ascending degree.

The three-arguments signature returns an algebraic extension of that field, with minimum polynomial equal to the [Conway polynomial](https://en.wikipedia.org/wiki/Conway_polynomial_(finite_fields))  for $(p,n)$. The `GaloisFields` package ships with a database of Conway  polynomials and will raise an error if it does not contain an entry for  $(p,n)$.

Note that in the latter two cases, the variable name (e.g. β above) is part of the type. This lets you define identifications between isomorphic (sub)fields. For example, with the following definition

```
const F = @GaloisField! 𝔽₂ β^2 + β + 1
const G = @GaloisField! 𝔽₂ γ^2 + γ + 1
```

the fields $F$ and $G$ are isomorphic, but not canonically. We might define

```
@GaloisFields.identify β => γ + 1
@GaloisFields.identify γ => β + 1
```

to allow for conversions like

```
G(β)
convert(F, γ + 1)
```

In the Conway case, you do not have to define your own identifications, as the Conway polynomials satisfy compatibility relations that allow us to use certain distinguished inclusions between them.


In [335]:
x=[rand(2) for i=1:10]

10-element Vector{Vector{Float64}}:
 [0.7248512048074602, 0.5956281398569836]
 [0.3423616342729472, 0.30990956468641717]
 [0.2220817188124009, 0.0919372195820698]
 [0.43783398880046787, 0.47016092384227326]
 [0.791725980123458, 0.7052410433796314]
 [0.09217476498061083, 0.12725939866504188]
 [0.45281383084093174, 0.39122464103589816]
 [0.33806263297107253, 0.9542242098549534]
 [0.5664683799053358, 0.4337947502526993]
 [0.04083282329424409, 0.40240658284532693]

In [344]:
x .+= tuple([1,1])

10-element Vector{Vector{Float64}}:
 [5.72485120480746, 5.595628139856983]
 [5.342361634272947, 5.309909564686417]
 [5.222081718812401, 5.09193721958207]
 [5.437833988800468, 5.470160923842274]
 [5.7917259801234575, 5.7052410433796314]
 [5.09217476498061, 5.127259398665042]
 [5.452813830840932, 5.391224641035898]
 [5.338062632971073, 5.954224209854953]
 [5.566468379905336, 5.4337947502527]
 [5.040832823294244, 5.402406582845327]